In [1]:
# default_exp inference

In [2]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# inference

> All inference related methods

In [3]:
#export
import datetime, warnings, gc
from inspect import signature
import sklearn.metrics as skm

from fastai2.text.all import *

from tritonlytics_ai.utils import *

In [4]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

In [5]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Utility 

In [6]:
#export
def concat_pool(raw_outputs):
    last_rnn_layer = raw_outputs[:,-1,None] # (e.g. (bs,n_hid,emb_sz) => (bs,1,emb_sz)) 
    bsz = last_rnn_layer.shape[0] 
    
    avg_pool = F.adaptive_avg_pool1d(last_rnn_layer.permute(0,2,1), 1).view(bsz, -1)
    max_pool = F.adaptive_max_pool1d(last_rnn_layer.permute(0,2,1), 1).view(bsz, -1)
    last_outp = last_rnn_layer[:,-1,:]

    return torch.cat([last_outp, max_pool, avg_pool], 1)

In [7]:
#export
def _get_verbatims_ulmfit_sentiment_preds(inf_df,
                                          learn_exports_fpath=SENTIMENT_CLS_PATH, 
                                          m_suf='multilabel', 
                                          backwards=False,
                                          corpus_cols=['answer_text'], 
                                          device=torch.device('cpu')):
    
    model_prefix = 'bwd' if backwards else 'fwd'
    
    # 1. grab learner, procs, and data
    inf_learn = load_learner(fname=learn_exports_fpath/f'{model_prefix}_export_clas_{m_suf}.pkl', cpu=False)
    inf_learn.model = inf_learn.model.to(device)
    inf_learn.model = inf_learn.model.eval()
    
    # 2. define a suitable dataloader
    tok_inf_df, tok_counts = tokenize_df(inf_df, corpus_cols)
    inf_dl = inf_learn.dls.test_dl(tok_inf_df, rm_type_tfms=None, bs=128)
    if (backwards): inf_dl.tfms.add(Transform(lambda nums: nums.flip(0)))

    # 3. get probs and document vectors
    test_probs, doc_vecs, concat_doc_vecs = [], [], []
    with torch.no_grad():
        for index, b in enumerate(inf_dl):
            if index % 1000 == 0:  pass #print(index)

            # reset hidden state (if you don't do this you will OOM)
            inf_learn.model.reset()
            
            # note: even though there is no targets, each batch is a tuple!
            probs, raw_outputs, outputs = inf_learn.model(b[0])
            
            # why "detach"? the computation of gradients wrt the weights of netG can be fully 
            # avoided in the backward pass if the graph is detached where it is.
            test_probs.append(to_detach(probs))
            doc_vecs.append(to_detach(raw_outputs[:,-1,:]))
            concat_doc_vecs.append(to_detach(concat_pool(raw_outputs)))

    all_probs = L(torch.cat(test_probs))
    all_vecs = L(torch.cat(doc_vecs))
    all_concat_vecs = L(torch.cat(concat_doc_vecs))

    # 4. ensure results are returned in order
    # test_dl.get_idxs() => unsorted/original order items
    all_probs = all_probs[0][np.argsort(inf_dl.get_idxs())]
    all_vecs = all_vecs[0][np.argsort(inf_dl.get_idxs())]
    all_concat_vecs = all_concat_vecs[0][np.argsort(inf_dl.get_idxs())]
        
    # 5. return ordered results
    inf_learn, inf_data = None, None; gc.collect()
    
    return all_probs, all_vecs, all_concat_vecs

In [8]:
#export
def _get_verbatims_ulmfit_standard_themes_saw(inf_df,
                                              learn_exports_fpath=STANDARD_THEME_SAW_PATH, 
                                              m_suf='multilabel', 
                                              backwards=False,
                                              corpus_cols=['answer_text'], 
                                              device=torch.device('cpu')):
    
    model_prefix = 'bwd' if backwards else 'fwd'
    
    # 1. grab learner, procs, and data
    inf_learn = load_learner(fname=learn_exports_fpath/f'{model_prefix}_export_clas_{m_suf}.pkl', cpu=False)
    inf_learn.model = inf_learn.model.to(device)
    inf_learn.model = inf_learn.model.eval()
    
    # 2. define a suitable dataloader
    tok_inf_df, tok_counts = tokenize_df(inf_df, corpus_cols)
    inf_dl = inf_learn.dls.test_dl(tok_inf_df, rm_type_tfms=None, bs=128)
    if (backwards): inf_dl.tfms.add(Transform(lambda nums: nums.flip(0)))

    # 3. get probs and document vectors
    test_probs, doc_vecs, concat_doc_vecs = [], [], []
    with torch.no_grad():
        for index, b in enumerate(inf_dl):
            if index % 1000 == 0: pass #print(index)

            # reset hidden state (if you don't do this you will OOM)
            inf_learn.model.reset()
            
            # note: even though there is no targets, each batch is a tuple!
            probs, raw_outputs, outputs = inf_learn.model(b[0])
            
            # why "detach"? the computation of gradients wrt the weights of netG can be fully 
            # avoided in the backward pass if the graph is detached where it is.
            test_probs.append(to_detach(probs))
            doc_vecs.append(to_detach(raw_outputs[:,-1,:]))
            concat_doc_vecs.append(to_detach(concat_pool(raw_outputs)))

    all_probs = L(torch.cat(test_probs))
    all_vecs = L(torch.cat(doc_vecs))
    all_concat_vecs = L(torch.cat(concat_doc_vecs))

    # 4. ensure results are returned in order
    # test_dl.get_idxs() => unsorted/original order items
    all_probs = all_probs[0][np.argsort(inf_dl.get_idxs())]
    all_vecs = all_vecs[0][np.argsort(inf_dl.get_idxs())]
    all_concat_vecs = all_concat_vecs[0][np.argsort(inf_dl.get_idxs())]
        
    # 5. return ordered results
    inf_learn, inf_data = None, None; gc.collect()
    
    return all_probs, all_vecs, all_concat_vecs

In [9]:
#export

# our multi-modal module for standard themes meta preds
class MM(Module):
    def __init__(self, in_features=50): 
        super().__init__()
        self.pred_is_example = nn.Linear(in_features, 2, bias=False)
        self.pred_avg_sentiment = nn.Linear(in_features, 1, bias=False)
        self.pred_avg_sent_range = SigmoidRange(1., 5.1)
        
    def forward(self, x):
        is_example = self.pred_is_example(x)
        avg_sentiment = self.pred_avg_sent_range(self.pred_avg_sentiment(x))
        
        return avg_sentiment, is_example
    
# "load_learner" requires our custom metrics used in training be present here
def sentiment_mse(preds, *targs): return mse(preds[0], targs[0])
def is_example_acc(preds, *targs): return accuracy(preds[1], targs[1])


def _get_verbatims_ulmfit_standard_themes_meta(inf_df,
                                               learn_exports_fpath=STANDARD_THEME_META_PATH, 
                                               m_suf='multitask', 
                                               backwards=False,
                                               corpus_cols=['theme', 'answer_text'], 
                                               device=torch.device('cpu')):
    
    model_prefix = 'bwd' if backwards else 'fwd'
    
    # 1. grab learner, procs, and data
    inf_learn = load_learner(fname=learn_exports_fpath/f'{model_prefix}_export_mm_{m_suf}.pkl', cpu=False)
    inf_learn.model = inf_learn.model.to(device)
    inf_learn.model = inf_learn.model.eval()
    
    # 2. define a suitable dataloader
    tok_inf_df, tok_counts = tokenize_df(inf_df, corpus_cols)
    inf_dl = inf_learn.dls.test_dl(tok_inf_df, rm_type_tfms=None, bs=128)
    if (backwards): inf_dl.tfms.add(Transform(lambda nums: nums.flip(0)))

    # 3. get probs
    test_probs_sent, test_probs_is_example = [], []
    with torch.no_grad():
        for index, b in enumerate(inf_dl):
            if index % 1000 == 0: pass #print(index)

            # reset hidden state (if you don't do this you will OOM)
            inf_learn.model.reset()
            
            # note: even though there is no targets, each batch is a tuple!
            probs, raw_outputs, outputs = inf_learn.model(b[0])
            
            # why "detach"? the computation of gradients wrt the weights of netG can be fully 
            # avoided in the backward pass if the graph is detached where it is.
            test_probs_sent.append(to_detach(probs[0]))
            test_probs_is_example.append(to_detach(probs[1]))

    all_probs_sent = L(torch.cat(test_probs_sent))
    all_probs_is_example = L(torch.cat(test_probs_is_example))

    # 4. ensure results are returned in order
    # test_dl.get_idxs() => unsorted/original order items
    all_probs_sent = all_probs_sent[0][np.argsort(inf_dl.get_idxs())]
    all_probs_is_example = all_probs_is_example[0][np.argsort(inf_dl.get_idxs())]

    # 5. return ordered results
    inf_learn, inf_data = None, None; gc.collect()
    
    return all_probs_sent, all_probs_is_example

## Inference methods

In [10]:
#export
def get_sentiment_preds(inf_df, learn_exports_fpath=SENTIMENT_CLS_PATH, m_suf='multilabel',
                        corpus_cols=['answer_text'], 
                        with_doc_vecs=True,
                        device=torch.device('cpu')):
    
    # ensemble forward and backwards models
    probs_fwd, vecs_fwd, concat_vecs_fwd = _get_verbatims_ulmfit_sentiment_preds(inf_df,
                                                                                 learn_exports_fpath,
                                                                                 m_suf=m_suf,
                                                                                 backwards=False,
                                                                                 corpus_cols=corpus_cols,
                                                                                 device=device)
    
    probs_bwd, vecs_bwd, concat_vecs_bwd = _get_verbatims_ulmfit_sentiment_preds(inf_df,
                                                                                 learn_exports_fpath,
                                                                                 m_suf=m_suf,
                                                                                 backwards=True,
                                                                                 corpus_cols=corpus_cols,
                                                                                 device=device)

    probs_final = torch.sigmoid((probs_fwd + probs_bwd) / 2)

    # grab the probabilities of each sentiment label; concatentate to input
    prob_labels = ['prob_' + lbl for lbl in SENT_LABELS[1:]]
    probs_df = pd.DataFrame(probs_final.numpy(), columns=prob_labels)
    
    final_df = pd.concat([inf_df, probs_df], axis=1)
    
    if (not with_doc_vecs): return final_df
    return final_df, vecs_fwd, vecs_bwd, concat_vecs_fwd, concat_vecs_bwd

In [11]:
#export
def get_standard_theme_saw_preds(inf_df, learn_exports_fpath=STANDARD_THEME_SAW_PATH, m_suf='multilabel',
                                 corpus_cols=['answer_text'],
                                 with_doc_vecs=True,
                                 device=torch.device('cpu')):
    
    # ensemble forward and backwards models
    probs_fwd, vecs_fwd, concat_vecs_fwd = _get_verbatims_ulmfit_standard_themes_saw(inf_df,
                                                                                     learn_exports_fpath,
                                                                                     m_suf=m_suf,
                                                                                     backwards=False,
                                                                                     corpus_cols=corpus_cols,
                                                                                     device=device)
    
    probs_bwd, vecs_bwd, concat_vecs_bwd = _get_verbatims_ulmfit_standard_themes_saw(inf_df,
                                                                                     learn_exports_fpath,
                                                                                     m_suf=m_suf,
                                                                                     backwards=True,
                                                                                     corpus_cols=corpus_cols,
                                                                                     device=device)

    probs_final = torch.sigmoid((probs_fwd + probs_bwd) / 2)

    # grab the probabilities of each sentiment label; concatentate to input
    prob_labels = ['prob_' + lbl for lbl in STANDARD_THEME_SAW_LABELS]
    probs_df = pd.DataFrame(probs_final.numpy(), columns=prob_labels)
    
    final_df = pd.concat([inf_df, probs_df], axis=1)
    
    # add metadata specific thresholds, accuracies, and combined loss
    with open(learn_exports_fpath/'model_results.json') as f: model_results = json.load(f)
    
    final_df['threshold_f05'] = model_results['threshold_f05']
    final_df['threshold_f1'] = model_results['threshold_f1']
    final_df['threshold_f2'] = model_results['threshold_f2']

    final_df['val_acc_f05'] = model_results['val_acc_f05']
    final_df['val_acc_f1'] = model_results['val_acc_f1']
    final_df['val_acc_f2'] = model_results['val_acc_f2']
    
    final_df['val_loss'] = model_results['val_loss']
    
    if (not with_doc_vecs): return final_df
    return final_df, vecs_fwd, vecs_bwd, concat_vecs_fwd, concat_vecs_bwd

In [12]:
#export
def get_standard_theme_preds(themes_df, learn_exports_fpath=STANDARD_THEME_META_PATH, m_suf='multitask',
                            corpus_cols=['theme', 'answer_text'],
                            device=torch.device('cpu')):
    
    # 1. we need to massage the "themes_df" so that we have one row per predicted theme (only keeping those
    # themes that meet the minimum f2 threshold)
    pred_theme_cols = filter_col = [col for col in themes_df if col.startswith('prob_')]
    
    inf_df = themes_df.melt(id_vars=list(TASK_LM_DTYPES_SC.keys()) + ['threshold_f05', 'threshold_f1', 'threshold_f2', 'val_acc_f05', 'val_acc_f1', 'val_acc_f2', 'val_loss'], 
                            value_vars=pred_theme_cols, 
                            var_name='theme', 
                            value_name='theme_prob')
    
    inf_df = inf_df.loc[inf_df.theme_prob >= inf_df.threshold_f2]
    
    inf_df['url_friendly_theme'] = inf_df.theme.apply(
        lambda s: re.sub("(.*?)_([a-zA-Z])","\g<1> \g<2>",s).replace('prob', '').strip().title().replace(' ',''))

    inf_df['theme'] = inf_df.url_friendly_theme.apply(lambda s: re.sub("([a-z])([A-Z])","\g<1> \g<2>",s))
    inf_df.reset_index(inplace=True)
    
    # 2. ensemble forward and backwards models
    preds_fwd_sent, probs_fwd_is_example = _get_verbatims_ulmfit_standard_themes_meta(inf_df,
                                                                                      learn_exports_fpath, 
                                                                                      m_suf=m_suf, 
                                                                                      backwards=False,
                                                                                      corpus_cols=corpus_cols,
                                                                                      device=device)
    
    preds_bwd_sent, probs_bwd_is_example = _get_verbatims_ulmfit_standard_themes_meta(inf_df,
                                                                                      learn_exports_fpath, 
                                                                                      m_suf=m_suf, 
                                                                                      backwards=True,
                                                                                      corpus_cols=corpus_cols,
                                                                                      device=device)

    preds_final_sent = (preds_fwd_sent + preds_bwd_sent) / 2
    probs_final_is_example = torch.softmax((probs_fwd_is_example + probs_bwd_is_example) / 2, dim=-1)[:,1]
    
    # add the probabilities of each label
    combined_probs = np.concatenate((preds_final_sent.numpy(), probs_final_is_example.numpy()[:,None]), axis=1)
    
    col_labels = ['pred_sentiment', 'prob_is_example']
    probs_df = pd.DataFrame(combined_probs, columns=col_labels)
    final_df = pd.concat([inf_df, probs_df], axis=1)
    
    # add metadata specific thresholds, accuracies, and combined loss
    with open(learn_exports_fpath/'model_results.json') as f: model_results = json.load(f)
    
    final_df['is_example_threshold_f05'] = model_results['is_example_threshold_f05']
    final_df['is_example_threshold_f1'] = model_results['is_example_threshold_f1']
    final_df['is_example_threshold_f2'] = model_results['is_example_threshold_f2']

    final_df['is_example_accuracy_f05'] = model_results['val_acc_f05']
    final_df['is_example_accuracy_f1'] = model_results['val_acc_f1']
    final_df['is_example_accuracy_f2'] = model_results['val_acc_f2']
    
    final_df['sentiment_mae'] = model_results['sentiment_mae']
    final_df['sentiment_mse'] = model_results['sentiment_mse']
    final_df['sentiment_rmse'] = model_results['sentiment_rmse']
    
    final_df['val_loss_metadata'] = model_results['final_valid_loss']
    
    return final_df

## Tests

In [13]:
f = json.loads((RAW_DATA_PATH/'verbatim-inference.json').read_text())

In [14]:
models_df = pd.DataFrame(f['models'])
models_df.head()

,model_run_id,survey_id,model_id,model_name,model_type_id,model_type_name
0,124,396,5,20200511-ulmfit-verbatim-standard-themes-saw,4,verbatim-classification-saw-themes
1,125,396,4,20200511-ulmfit-verbatim-sent,2,verbatim-classification-sentiment


In [15]:
inf_df = pd.DataFrame(f['data'])
print(len(inf_df));

698


In [16]:
yyyymmdd = datetime.today().strftime("%Y%m%d")

device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:1


### Sentiment

In [17]:
res = get_sentiment_preds(inf_df, device=device)

print(len(res[0]))
print(res[3].shape, res[1].shape)
print(res[4].shape, res[2].shape)

res[0].head(2)
# res[0].to_csv(f'~/inf_X_sentiment.csv', index=False)

698
torch.Size([698, 1200]) torch.Size([698, 400])
torch.Size([698, 1200]) torch.Size([698, 400])


,id,survey_id,question_ans_id,rsp_id,question_category_id,group_id,answer_text,answer_text_non_english,language,survey_type_id,...,group_level8_code,group_level8_name,prob_is_very_positive,prob_is_positive,prob_is_very_negative,prob_is_negative,prob_is_suggestion,prob_feels_threatened,prob_has_profanity,prob_is_nonsense
0,660404,396,93069,480447,1141,6988,"I believe ANR leadership is completely disconnected from the work of campus-based specialists. When I am responding regarding ""my department"" in this survey it is my campus department I am evaluating, not an ANR department.",None,English,47,...,None,None,0.000215,0.006649,0.102388,0.911618,0.006636,0.000759,0.000005,0.010853
1,659993,396,92963,480435,1155,6958,Lorena Hoyos\r\nUCCE SAN JOAQUIN COUNTY\r\nUC CALFRESH,None,English,47,...,None,None,0.012533,0.017884,0.002087,0.001468,0.005406,0.000027,0.000016,0.045634


### Standard Themes - S@W

In [18]:
res = get_standard_theme_saw_preds(inf_df, device=device)

print(len(res[0]))
print(res[3].shape, res[1].shape)
print(res[4].shape, res[2].shape)

res[0].head(2)

698
torch.Size([698, 1200]) torch.Size([698, 400])
torch.Size([698, 1200]) torch.Size([698, 400])


,id,survey_id,question_ans_id,rsp_id,question_category_id,group_id,answer_text,answer_text_non_english,language,survey_type_id,...,prob_salary_pay,prob_satisfied_with_diversity_progams,prob_supervisor_effectiveness_resolves_staff_issues,threshold_f05,threshold_f1,threshold_f2,val_acc_f05,val_acc_f1,val_acc_f2,val_loss
0,660404,396,93069,480447,1141,6988,"I believe ANR leadership is completely disconnected from the work of campus-based specialists. When I am responding regarding ""my department"" in this survey it is my campus department I am evaluating, not an ANR department.",None,English,47,...,0.011179,0.026293,0.000754,0.49,0.44,0.27,0.976194,0.976055,0.966021,0.096622
1,659993,396,92963,480435,1155,6958,Lorena Hoyos\r\nUCCE SAN JOAQUIN COUNTY\r\nUC CALFRESH,None,English,47,...,0.001565,0.005855,0.093966,0.49,0.44,0.27,0.976194,0.976055,0.966021,0.096622


### Standard Themes Metadata - S@W

In [19]:
themes_df = get_standard_theme_saw_preds(inf_df, device=device, with_doc_vecs=False)

res = get_standard_theme_preds(themes_df, device=device)

print(len(res))
res.head(2)
# res.to_csv(f'~/inf_X_standard_themes.csv', index=False)

1246


,index,id,question_ans_id,answer_text,answer_text_non_english,language,survey_id,survey_type_id,benchmark_survey_type,client_id,...,is_example_threshold_f05,is_example_threshold_f1,is_example_threshold_f2,is_example_accuracy_f05,is_example_accuracy_f1,is_example_accuracy_f2,sentiment_mae,sentiment_mse,sentiment_rmse,val_loss_metadata
0,11,659998,92963,"Anna Martin at NPI is an absolute joy to work for. She makes an effort to show us her appreciation for everything we do, even the ""behind-the-scenes"" work that can easily go unrecognized. She routinely asks about our capacity and workloads. Lorrene Richie and other staff (namely Anna) have turned NPI into a model for flexibility and work-life balance. It's no secret our program is female-dominated, many of those being mothers of young children. Lorrene always champions our efforts and it's encourgaing to stay in the workforce.",None,English,396,47,SAW,UCANR,...,0.57,0.57,0.57,0.976298,0.976298,0.976298,0.458426,0.383636,0.619384,0.498877
1,18,660413,93069,"While our department has a good work environment as a whole there are several area's that are a major issues.\r\n1) We have zero leadership, we have no direct managers over us. While we are able to manage our own work, we have very little real direction or support and it directly ties into other issues here.\r\n2) There is a Lone Ranger in our department. This employee has control over the servers without oversight and without transparency. This person has taken advantage of their role and privileged access to assert authority over others when this person has none. There is no collaboratio...",None,English,396,47,SAW,UCANR,...,0.57,0.57,0.57,0.976298,0.976298,0.976298,0.458426,0.383636,0.619384,0.498877


In [20]:
print(res[res.id == 660413].answer_text.values[0])
print(res[res.id == 660413].theme)
print(res[res.id == 660413].pred_sentiment)

While our department has a good work environment as a whole there are several area's that are a major issues.\r\n1) We have zero leadership, we have no direct managers over us. While we are able to manage our own work, we have very little real direction or support and it directly ties into other issues here.\r\n2) There is a Lone Ranger in our department. This employee has control over the servers without oversight and without transparency. This person has taken advantage of their role and privileged access to assert authority over others when this person has none. There is no collaboration with others. This person may destroy ANR IT completely, causing many to desire to leave and find work elsewhere. There are MANY other serious issues which I will not share here...\r\n3) As an independent business unit. ANR seeks to bring on additional services which require entire teams to be hired. Yet the prospect seems like little growth for teams already over capacity.\r\n4) We are unable to tak

## Cleanup

In [21]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 99_inference.ipynb.
Converted index.ipynb.
